In [1]:
!pip install nibabel
!pip install glob
!pip install nilearn

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [0]:
#importing required libraries
import os
import numpy as np
import nibabel as nib
import glob
import torch

In [0]:
#defining path for required files
path_1 = glob.glob('/content/drive/My Drive/selected_1mm^3/*')
path_112 = glob.glob('/content/drive/My Drive/selected_112mm^3/*')

In [0]:
#read images from paths
images_isotropic = []
images_downsampled = []

for images in path_1:
  data_isotropic = nib.load(images)
  data_isotropic = data_isotropic.get_fdata()
  images_isotropic.append(data_isotropic)

for images in path_112:
  data_downsampled = nib.load(images)
  data_downsampled = data_downsampled.get_fdata()
  images_downsampled.append(data_downsampled)


In [6]:
#verifying shapes of isotropic and downsampled images
print(images_isotropic[1].shape) 
print(images_downsampled[1].shape) 

(239, 239, 162)
(239, 239, 81)


In [0]:
#get arrays for isotropic and downsampled images
array_images_isotropic = np.asarray(images_isotropic)
array_images_downsampled = np.asarray(images_downsampled)

In [8]:
print(array_images_isotropic.shape)
print(array_images_downsampled.shape)

(2, 239, 239, 162)
(2, 239, 239, 81)


In [11]:
 # getting images in BCDHW format
labels = []
input_images = []
for x,y in zip(array_images_isotropic,array_images_downsampled):
  swapped_labels = np.moveaxis(x, 2, 0)
  swapped_input_images = np.moveaxis(y, 2, 0)
  labels_expand = np.expand_dims(swapped_labels, 0)
  labels_expand = np.expand_dims(labels_expand, 0)
  input_expand = np.expand_dims(swapped_input_images, 0)
  input_expand = np.expand_dims(input_expand, 0)
  labels.append(labels_expand)
  input_images.append(input_expand)
print(len(labels), labels[1].shape)
print(len(input_images),input_images[1].shape)

2 (1, 1, 162, 239, 239)
2 (1, 1, 81, 239, 239)


In [0]:
array_labels = np.asarray(labels)
array_input_images = np.asarray(input_images)

In [0]:
#split Train and Test data for validation
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(array_input_images, array_labels, test_size=0.1)

In [0]:
X_train = np.asarray(X_train)
X_train = X_train.astype(np.float64)
X_test = np.asarray(X_test)
X_test = X_test.astype(np.float64)
Y_train = np.asarray(Y_train)
Y_train = Y_train.astype(np.float64)
Y_test = np.asarray(Y_test)
Y_test = Y_test.astype(np.float64)

In [15]:
#converting numpy array to torch tensors
torch_x_train = torch.from_numpy(X_train)
torch_y_train = torch.from_numpy(Y_train)
torch_x_test = torch.from_numpy(X_test)
torch_y_test = torch.from_numpy(Y_test)
print(torch_x_train.shape,torch_y_train.shape)
print(torch_x_test.shape,torch_y_test.shape)

torch.Size([1, 1, 1, 81, 239, 239]) torch.Size([1, 1, 1, 162, 239, 239])
torch.Size([1, 1, 1, 81, 239, 239]) torch.Size([1, 1, 1, 162, 239, 239])


In [16]:
#mapping train data in train_loader
train_loader = []
for i, j in zip(torch_x_train, torch_y_train):
    train = (i,j)
    train_loader.append(train)
print(len(train_loader))

1


In [0]:
#mapping test data in test_loader
test_loader = []
for i, j in zip(torch_x_test, torch_y_test):
    test = (i,j)
    test_loader.append(test)

In [0]:
#deleting unwanted variables
del(path_1, path_112, images_isotropic,images_downsampled,array_images_isotropic,array_images_downsampled,labels,input_images)
del(array_labels,array_input_images,X_train,X_test,Y_train,Y_test)
del(torch_x_train,torch_x_test,torch_y_train,torch_y_test)

In [0]:
import torch
import torch.nn as nn

In [0]:
def create_conv_bn_relu(in_channels, out_channels, kernel_size=3, padding=1):
    return nn.Sequential(
        nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.BatchNorm3d(out_channels),
        nn.ReLU(inplace=True),
    )

def create_double_conv(in_channels, mid_channels, out_channels, kernel_size=3, padding=1):
    return nn.Sequential(
        create_conv_bn_relu(in_channels, mid_channels, kernel_size, padding),
        create_conv_bn_relu(mid_channels, out_channels, kernel_size, padding),
    )

def upsample(x1):
    return torch.nn.functional.interpolate(
        x1,
        size=(2*(x1.size()[2]), x1.size()[3], x1.size()[4]),
        scale_factor=None,
        mode='trilinear',
        align_corners=True
    )

def upsample_1(x1, x2):
    return torch.nn.functional.interpolate(
        x1,
        size=(x2.size()[2], x2.size()[3], x2.size()[4]),
        scale_factor=None,
        mode='trilinear',
        align_corners=True
    )
 
class UNet(nn.Module):
    def __init__(self, n_channels, output_channels):
        super(UNet, self).__init__()

        self.n_channels = n_channels
        self.output_channels = output_channels

        self.double_conv_down_0 = create_double_conv(1, 10, 10)
        self.double_conv_down_1 = create_double_conv(10, 10, 10)
        self.double_conv_down_2 = create_double_conv(10, 10, 10)
        self.double_conv_down_3 = create_double_conv(10, 10, 10)
        self.double_conv_down_4 = create_double_conv(10, 10, 10)

        self.maxpool_1 = nn.MaxPool3d((1, 2, 2))
        self.maxpool_2 = nn.MaxPool3d((1, 2, 2))
        self.maxpool_3 = nn.MaxPool3d((1, 2, 2))
        self.maxpool_4 = nn.MaxPool3d((1, 2, 2))

        self.double_conv_up_1 = create_double_conv(20, 10, 10)
        self.double_conv_up_2 = create_double_conv(20, 10, 10)
        self.double_conv_up_3 = create_double_conv(20, 10, 10)
        self.double_conv_up_4 = create_double_conv(20, 10, 10)

        self.output_conv = nn.Conv3d(1, 1, kernel_size=1)

    def forward(self, x):
        
        x = upsample(x)
        x1 = self.double_conv_down_0(x)
        x = self.maxpool_1(x1)
        x2 = self.double_conv_down_1(x)
        x = self.maxpool_2(x2)
        x3 = self.double_conv_down_2(x)
        x = self.maxpool_3(x3)
        x4 = self.double_conv_down_3(x)
        x = self.maxpool_4(x4)
        x5 = self.double_conv_down_4(x)

        x = upsample_1(x5, x4)
        x = torch.cat([x, x4], dim=1)
        x = self.double_conv_up_1(x)

        x = upsample_1(x, x3)
        x = torch.cat([x, x3], dim=1)
        x = self.double_conv_up_2(x)

        x = upsample_1(x, x2)
        x = torch.cat([x, x2], dim=1)
        x = self.double_conv_up_3(x)

        x = upsample_1(x, x1)
        x = torch.cat([x, x1], dim=1)
        x = self.double_conv_up_4(x)

        output = self.output_conv(x)

        return output

In [0]:
model = UNet(10,10)

In [0]:
loss_function = nn.BCEWithLogitsLoss()

In [0]:
import torch.optim as optim

In [0]:
#looping training data for training
for epoch in range(2):  

    running_loss = 0.0

    for i,[inputs, labels] in enumerate(train_loader):
        outputs = model(inputs.float())
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        optimizer.zero_grad()
        loss = loss_function(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[{}, {}] loss: {}'.format(epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

       
        """if i % 10000 == 0:
            torch.save(model.state_dict(), <PATH>)"""

        if i % 10000 == 0:
            test_loss = []
            for inputs, labels in test_loader: # TODO Add test_loader
                outputs = model(inputs.float())
                loss = loss_function(outputs, labels.float())
                # we do not update model here
                test_loss.append(loss)
            print ('Loss on test set = {}'.format(sum(test_loss) / len(test_loss)))